In [1]:
import tensorflow as tf
import tensorflow.keras as K
from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt


2023-11-29 08:40:06.095185: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-29 08:40:06.097042: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 08:40:06.129674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 08:40:06.129712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 08:40:06.130568: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
# Load and preprocess ISIC data
def load_isic_data(image_directory, metadata_path):
    metadata = pd.read_csv(metadata_path)
    images = []
    labels = []

    for i, row in metadata.iterrows():
        img_path = os.path.join(image_directory, row["isic_id"] + ".JPG")
        img = image.load_img(img_path, target_size=(224, 224))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        images.append(img)
        labels.append(row["diagnosis"])  # Assuming 'label' column exists

    X = np.vstack(images)
    Y = np.array(labels)
    return X, Y


In [6]:
def preprocess_data(X, Y):
    X = K.applications.densenet.preprocess_input(X)

    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(Y)
    Y = K.utils.to_categorical(encoded_labels, num_classes=8)

    # Y = K.utils.to_categorical(Y, num_classes=8)
    return X, Y, label_encoder


In [4]:
# Load your data
image_directory = "../data/images"
metadata_path = "../data/ham10000_metadata_2023-11-27.csv"
os.getcwd()

'/uw/ml_unsuper/ISIC_proc/src'

In [5]:
X, Y = load_isic_data(image_directory, metadata_path)
X, Y, label_encoder = preprocess_data(X, Y)


In [7]:

# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)


In [8]:
input_tensor = K.Input(shape=(224, 224, 3))  # Adjusted to match the shape of your preprocessed images

model = K.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_tensor=input_tensor,  # Directly use the input_tensor
    pooling="max",
    classes=1000,
)

2023-11-29 08:43:05.472307: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-11-29 08:43:05.472349: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: ZeroSubstance
2023-11-29 08:43:05.472352: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: ZeroSubstance
2023-11-29 08:43:05.472422: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 525.147.5
2023-11-29 08:43:05.472434: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 525.147.5
2023-11-29 08:43:05.472436: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 525.147.5


In [9]:

# %%
for layer in model.layers:
    layer.trainable = False
output = model.layers[-1].output
flatten = K.layers.Flatten(name="feats")
output = flatten(output)
model = K.models.Model(inputs=input_tensor, outputs=output)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']      

In [10]:

# %%
y_test.shape

# %%
preds_train = model.predict(x_train)
preds_train.shape


293/293 [==============================] - 238s 801ms/step


(9376, 1920)

In [11]:

# %%
preds_test = model.predict(x_test)
preds_test.shape


74/74 [==============================] - 63s 802ms/step


(2344, 1920)

In [12]:
from sklearn.cluster import KMeans

kmeans = KMeans(
    n_clusters=8, random_state=42, max_iter=1000, algorithm="elkan", tol=0.000001
).fit(preds_test)


/uw/.venvs/ml_research/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [13]:
from collections import Counter

def view_data(y1, preds, mapping):
    # Initialize dictionaries to store the results
    cluster_stats = {}
    totals = {key: 0 for key in range(8)}
    max_values = {key: {'label': None, 'value': 0} for key in range(8)}

    for val in set(y1):
        inds = [i for i in range(len(y1)) if y1[i] == val]
        p = preds[inds]
        y2 = y1[inds]
        counts = dict(Counter(p))

        # Store the counts in the cluster_stats dictionary
        cluster_stats[val] = counts

        # Calculate totals and update max_values for each key (cluster)
        for key, value in counts.items():
            totals[key] += value
            if value > max_values[key]['value']:
                max_values[key] = {'label': key, 'value': value}

        print("Cluster:", val)
        print("Counts:", counts)
        # Max value and label
        print("Max value:", max(counts.values()))
        max_label = max(counts, key=counts.get)
        print("Max label:", max_label, mapping[max_label])
        # Total count
        print("Total count:", sum(counts.values()))
        print("----------------")

    # After processing all clusters, print the aggregated statistics
    print("Total Counts for Each Cluster:", totals)
    print("Maximum Value and Label for Each Cluster:", max_values)

    return cluster_stats, totals, max_values

# Usage of the function (assuming preds and y1 are defined)
# cluster_stats, totals, max_values = view_data(y1, preds)


In [14]:
y1 = np.argmax(y_train, axis=1)
preds = kmeans.predict(preds_train)

decoded_labels = label_encoder.inverse_transform(preds)



mapping = {}
reverse_mapping = {}

for d, p in zip(decoded_labels, preds):
    mapping[d] = p
    reverse_mapping[p] = d
    if len(mapping) == 8:
        break

cluster_stats, totals, max_values = view_data(y1, preds, reverse_mapping)
mapping

Cluster: 0
Counts: {7: 19, 5: 6, 6: 27, 2: 11, 3: 51, 1: 4}
Max value: 51
Max label: 3 melanoma
Total count: 118
----------------
Cluster: 1
Counts: {5: 104, 1: 44, 2: 80, 7: 72, 3: 99, 6: 74, 0: 19, 4: 3}
Max value: 104
Max label: 5 pigmented benign keratosis
Total count: 495
----------------
Cluster: 2
Counts: {7: 23, 6: 19, 3: 34, 2: 22, 5: 10, 1: 4, 4: 12, 0: 5}
Max value: 34
Max label: 3 melanoma
Total count: 129
----------------
Cluster: 3
Counts: {7: 81, 1: 215, 5: 119, 0: 67, 3: 186, 2: 221, 6: 115, 4: 17}
Max value: 221
Max label: 2 dermatofibroma
Total count: 1021
----------------
Cluster: 4
Counts: {2: 300, 0: 804, 5: 687, 4: 1032, 3: 1238, 1: 812, 7: 1055, 6: 280}
Max value: 1238
Max label: 3 melanoma
Total count: 6208
----------------
Cluster: 5
Counts: {6: 152, 5: 112, 3: 276, 1: 147, 7: 113, 4: 37, 2: 198, 0: 37}
Max value: 276
Max label: 3 melanoma
Total count: 1072
----------------
Cluster: 6
Counts: {6: 62, 3: 55, 7: 40, 1: 11, 0: 4, 2: 14, 5: 9}
Max value: 62
Max lab

{'pigmented benign keratosis': 5,
 'squamous cell carcinoma': 6,
 'dermatofibroma': 2,
 'actinic keratosis': 0,
 'nevus': 4,
 'melanoma': 3,
 'basal cell carcinoma': 1,
 'vascular lesion': 7}

In [15]:
cluster_stats
xxx = {}
for i in range(8):
    # get max value and label for for cluster_stats[i]
    max_value = max(cluster_stats[i].values())
    max_label = [k for k, v in cluster_stats[i].items() if v == max_value][0]
    xxx[i] = (reverse_mapping[max_label], max_value,)

xxx
   
    
    

{0: ('melanoma', 51),
 1: ('pigmented benign keratosis', 104),
 2: ('melanoma', 34),
 3: ('dermatofibroma', 221),
 4: ('melanoma', 1238),
 5: ('melanoma', 276),
 6: ('squamous cell carcinoma', 62),
 7: ('pigmented benign keratosis', 41)}

In [16]:
# get the max value and label for each cluster
import numpy as np
def get_max_value_label(cluster_labels, cluster_values):
    max_value = np.max(cluster_values)
    max_label = cluster_labels[np.argmax(cluster_values)]
    return max_value, max_label, cluster_values, cluster_labels

max_value, max_label, cluster_values, cluster_labels = get_max_value_label(cluster_stats, totals)

In [17]:
max_label


{7: 19, 5: 6, 6: 27, 2: 11, 3: 51, 1: 4}

In [18]:

# %%
# dense net
mapping = {0: 4, 1: 7, 2: 0, 3: 2, 4: 3, 5: 5, 6: 9, 7: 8, 8: 6, 9: 7}
correct = 0
for val in set(y1):
    inds = [i for i in range(len(y1)) if y1[i] == val]  # all indices
    p = preds[inds]
    y2 = y1[inds]
    counts = dict(Counter(p))
    correct += counts[mapping[y2[0]]]
print("accuracy: ", correct * 100 / len(y1))

# %%


KeyError: 4

In [ ]:
import tensorflow as tf
import tensorflow.keras as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def preprocess_data(X, Y):
    X = K.applications.densenet.preprocess_input(X)
    Y = K.utils.to_categorical(Y)
    return X, Y

In [ ]:
(x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

In [ ]:
initializer = K.initializers.he_normal()
input_tensor = K.Input(shape=(32, 32, 3))

# resized_images = K.layers.Lambda(lambda image: tf.image.resize(image, (224, 224)))(input_tensor)
model = K.applications.DenseNet201(include_top=False,
                                   weights='imagenet',
                                   input_tensor=resized_images,
                                   input_shape=(224, 224, 3),
                                   pooling='max',
                                   classes=1000)

In [ ]:
for layer in model.layers:
    layer.trainable = False
output = model.layers[-1].output
flatten = K.layers.Flatten(name='feats')
output = flatten(output)
model = K.models.Model(inputs=input_tensor, outputs=output)
model.summary()

In [ ]:
y_test.shape

In [ ]:
preds=model.predict(x_train)
preds.shape

In [ ]:
preds_test=model.predict(x_test)
preds_test.shape

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=42, max_iter=1000, algorithm='elkan', tol=0.000001).fit(preds_test)

preds=kmeans.predict(preds_test) # use features extracted from denseNet
preds.shape

In [ ]:
y1=np.argmax(y_test,axis=1)
y1.shape

In [ ]:
# assign predicted clusters to actual classes to get the accuracy
from collections import Counter

correct=0
mapping={}

for val in set(y1):
  inds=[i for i in range(len(y1)) if y1[i]==val] # all indices for a particular class
  p=preds[inds] # predictions made for that class
  y2=y1[inds]
  counts=dict(Counter(p))

  print("y2: ", y2[:2])
  print("counts: ", counts)
  print("----------------")

In [ ]:
#dense net
mapping={
    0:4,
    1:7,
    2:0,
    3:2,
    4:3,
    5:5,
    6:9,
    7:8,
    8:6,
    9:7
}
correct=0
for val in set(y1):
  inds=[i for i in range(len(y1)) if y1[i]==val] # all indices
  p=preds[inds]
  y2=y1[inds]
  counts=dict(Counter(p))
  correct+=counts[mapping[y2[0]]]
print("accuracy: ", correct*100/len(y1))